In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path = '/kaggle/input/taxis-locations/Eindhoven Taxi Locations.xlsx - Blad1.csv'
data_ori = pd.read_csv(path)
data = data_ori.copy()

In [ ]:
data

In [ ]:
for i in data.index:
    data.loc[i, 'Lat'] = data.loc[i,'Lat, Lon'].split(',')[0]
    data.loc[i, 'Lon'] = data.loc[i,'Lat, Lon'].split(',')[1]
data = data.drop('Lat, Lon', axis =1)
data

In [ ]:
from sklearn.preprocessing import OneHotEncoder
OHE = OneHotEncoder()
OHE_col = OHE.fit_transform(data[['Taxi Location']])
OHE_col.index = data.index
encoded_taxi_loc = pd.DataFrame(OHE_col.toarray(), columns=OHE.get_feature_names_out(['Taxi Location']))
data = data.drop('Taxi Location', axis = 1)
data = pd.concat([data, encoded_taxi_loc], axis = 1)
encoded_taxi_loc
data

In [ ]:
data.dtypes
data['Lon'] = data['Lon'].astype('float64')
data['Lat'] = data['Lat'].astype('float64')
#data.dtypes

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
X = data[['Lat', 'Lon']]
y = data['Queue size']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0,test_size=0.2, train_size=0.8)
model = RandomForestRegressor(n_estimators =100)
model.fit(X_train, y_train)
y_preds = model.predict(X_test)
mean_absolute_error(y_preds, y_test) #1.77

In [ ]:
#predictions pour la ou le client se trouve
X_customer = pd.Series([51.444366, 5.470308],index = ['Lat', 'Lon'], name = 'coordonees')
predictions = model.predict([X_customer])
predictions = pd.Series(predictions, index = ['queue'], name = 'queue')
predictions 

In [ ]:
#predictions pour l'endroit ou les taxis se trouvent
#il me faudrait des donnees horaire
#si j'ai les donnes geographiques des endroits ou on peut creer des places de parking 
#pour les taxis, je pourrait predire la queue

In [ ]:
import numpy as np
# #j'ai toute les donnes du client (ou est ce qu'il est), je n'ai pas la queue
#calculer distance euclidienne
##X_customer = Lat, Lon

euclid_distance = pd.Series(111000*np.sqrt((data['Lon']-X_customer['Lon'])**2 + (data['Lat']-X_customer['Lat'])**2), name='distance')
print(euclid_distance)
customer_dataset = pd.concat([X_customer,  predictions], axis =0)
data_accessed_by_customer = pd.concat([data_ori,euclid_distance], axis =1)
data_accessed_by_customer.sort_values(by=['Queue size', 'distance'], inplace=True) #trier les choix par queue, puis distance
data_accessed_by_customer

In [ ]:
#optimize the model
from sklearn.model_selection import GridSearchCV, train_test_split

param_grid = {
    'n_estimators': [100, 200, 300],       # Nombre d'arbres dans la forêt
    'max_depth': [None, 10, 20, 30],       # Profondeur maximale des arbres
    'min_samples_split': [2, 5, 10],      # Nombre minimal d'échantillons requis pour diviser un nœud
    'min_samples_leaf': [1, 2, 4],        # Nombre minimal d'échantillons requis pour chaque feuille
    'max_features': ['auto', 'sqrt', 'log2']  # Nombre maximal de fonctionnalités à considérer lors de la division
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

mean_absolute_error(y_pred, y_test)